In [ ]:
import tensorflow as tf
print(tf.__version__)
import numpy as np
import cv2
from sklearn.preprocessing import StandardScaler
from time import sleep
import os
import cvzone
from cvzone.SelfiSegmentationModule import SelfiSegmentation

In [ ]:
classes = [
    'Doing other things',
    'Swiping Right',
    'Swiping Left',
    'Zooming In With Two Fingers',
    'Zooming Out With Two Fingers',
    'Stop Sign',
    'No gesture'
]

In [ ]:
classes[1]

In [ ]:
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])

In [ ]:
def normaliz_data(np_data):
    # Normalisation
    scaler = StandardScaler()
    #scaled_images  = normaliz_data2(np_data)
    scaled_images  = np_data.reshape(-1, 30, 64, 64, 1)
    return scaled_images

In [ ]:
def normaliz_data2(v):
    normalized_v = v / np.sqrt(np.sum(v**2))
    return normalized_v

In [ ]:
class Conv3DModel(tf.keras.Model):
    def __init__(self):
        super(Conv3DModel, self).__init__()
        self.conv1 = tf.compat.v2.keras.layers.Conv3D(32, (3, 3, 3), activation='relu', name="conv1", data_format='channels_last')
        self.pool1 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2), data_format='channels_last')
        self.conv2 = tf.compat.v2.keras.layers.Conv3D(64, (3, 3, 3), activation='relu', name="conv1", data_format='channels_last')
        self.pool2 = tf.keras.layers.MaxPool3D(pool_size=(2, 2,2), data_format='channels_last')
        self.convLSTM =tf.keras.layers.ConvLSTM2D(40, (3, 3))
    #norm
        self.flatten =  tf.keras.layers.Flatten(name="flatten")

    # Dense layers
        self.d1 = tf.keras.layers.Dense(128, activation='relu', name="d1")
        self.out = tf.keras.layers.Dense(7, activation='softmax', name="output")
        
        
    def call(self, x):
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.convLSTM(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.out(x)

In [ ]:
model1 = Conv3DModel()
model1.compile(loss='sparse_categorical_crossentropy',
                  optimizer=tf.keras.optimizers.RMSprop())

In [ ]:
model2 = Conv3DModel()
model2.compile(loss='sparse_categorical_crossentropy',
                  optimizer=tf.keras.optimizers.RMSprop())

In [ ]:
model1.load_weights('D:/3D-CNN/D/FinalModel/3D-CNN_LSTM_weights')

In [ ]:
model2.load_weights('D:/3D-CNN/D/All_Models/3D-CNN_LSTM_Adv_weights')

In [ ]:
font = cv2.FONT_HERSHEY_SIMPLEX

In [ ]:
class switch_1():
    def index_threshhold(conf, classes , pre):
        if classe == 0:
            print(conf[0])
            if pre == 0:
                return(classe)
            else:
                return(5)
        elif classe == 1:
            print(conf[0])
            if conf[0] >0.85:
                return(classe)
            else:
                return(6)
        elif classe ==  2 :
            print(conf[0])
            if conf[0] >0.85:
                return(classe)
            else:
                return(6)
        elif classe == 3:
            print(conf[0])
            if conf[0] >0.85:
                return(classe)
            else:
                return(6)
        elif classe == 4:
            print(conf[0])
            if conf[0] >0.85:
                return(classe)
            else:
                return(6)
        elif classe == 5:
            print(conf[0])
            if conf[0] >0.85:
                return(classe)
            else:
                return(6)
        elif classe == 6:
                return(6)
    def puttext_on(conf, index, classes,frame, font):   
        cv2.putText(frame, 'confidence: ' + str(conf), (10, 90), font, 0.7, (56, 71, 255), 2, 1)
        cv2.putText(frame, 'gesture class: ' + classe , (10, 60), font, 0.7, (56, 71, 255), 2, 1)

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
# Control for PPT:

import time
import pyautogui
to_predict = []
num_frames = 0
cap = cv2.VideoCapture(0)
cap.set(3, 640)
cap.set(4, 480)
pTime = 0
cTime = 0
cap_region_x_begin = 0.5  
cap_region_y_end = 0.8  
threshold = 60  
blurValue = 41 
bgSubThreshold = 50
learningRate = 0
pre = 0
conf = 0

segmentor = SelfiSegmentation()
classe =''
while(True):
    # Capture frame-by-frame
    ret, frame = cap.read()
    bgModel = cv2.createBackgroundSubtractorMOG2(0, bgSubThreshold)
    fgmask = bgModel.apply(frame)
    res = cv2.bitwise_and(frame, frame, mask=fgmask)
    gray = cv2.cvtColor(res, cv2.COLOR_BGR2GRAY)
    to_predict.append(cv2.resize(gray, (64, 64)))
    
    
        
    if len(to_predict) == 30:
        frame_to_predict = np.array(to_predict, dtype=np.float32)
        frame_to_predict = normaliz_data(frame_to_predict)
        result_1 = model1.predict(frame_to_predict)
        result_2 = model2.predict(frame_to_predict)
        result_3 = model3.predict(frame_to_predict)
        result_4 = model4.predict(frame_to_predict)
        
        # print(result)
        classe = classes[np.argmax(result_1+result_2)]
        predict = np.amax((result_1+result_2)/2)
        print(classe,(predict)*100,'%')
        if(classe == "Swiping Left"):
                pyautogui.press('left', presses = 1)

        elif(classe == "Swiping Right"):
                pyautogui.press('right', presses = 1)

        elif(classe == "Zooming In With Two Fingers"):
                pyautogui.hotkey('shift','f5')
                
        elif(classe == "Zooming Out With Two Fingers"):
                pyautogui.press('esc')
                
        elif(classe == "Doing other things"):
                pyautogui.press('None')

        else:
            pyautogui.press('None')

        to_predict = []
        real_index = switch_1.index_threshhold(predict, classe ,pre)
        # instruction = switch_1.controll_PC(real_index)
        pre = classes[0]
    switch_1.puttext_on(predict, real_index, classes[6], frame, font)
    cv2.putText(frame, 'FPS: ' + str(int(fps)), (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (56, 71, 255), 2, 1)
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime    
    cv2.imshow('Hand Gesture Recognition',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release
cv2.destroyAllWindows()